# 预处理

## 导包

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import time
import math
import datetime

## 通用API

In [ ]:
# 通用api
# 清除字段首尾的空白字符
def remove_space(x):
    if type(x) == str:
        x = x.strip()
        if len(x) == 0:
            return np.NaN
    return x

# 更新二级码
def update_table(update_df):
    f_data.loc[:, WH_HOUSE_USE] = f_data.loc[:, WH_HOUSE_USE].combine_first(update_df[WH_HOUSE_USE])

#读取原始数据
def get_r_data(r_data_file_path):
    result = []
    file_path = os.listdir(r_data_file_path)
    for path in file_path:
        data = pd.read_excel(r_data_file_path + path, dtype=object)
        result.append(data)
    r_data = pd.concat(result, ignore_index=True)
    r_data.index = r_data[HOUSE_ID]
    r_data.replace(r"\N", np.NaN, inplace=True)
#     r_data[HOUSE_USE] = r_data[HOUSE_USE].astype(int)
    r_data = r_data.loc[:, ~r_data.columns.str.contains('Unnamed')]
    return r_data

#读取过程数据
def get_data(file_path, dir_name):
    result = []
    for path in file_path:
        data = pd.read_excel(dir_name + path, dtype=object)
        result.append(data)
    return pd.concat(result, ignore_index=True)

#更新幢号、所在层、地上层
def update_code(FILE_DIR,DATA,code):
    if os.listdir(FILE_DIR):
        file_path = os.listdir(FILE_DIR)
        file_data = get_data(file_path, FILE_DIR)
        file_data.index = file_data[HOUSE_ID]
        DATA.loc[file_data[HOUSE_ID],
            code] = file_data[code]
        print (code+'已更新')
    else:
        print(code+'无需更新')
        
#导出人工清洗数据
def output_ex_data(data, fields, path, code):
    print(code+" 总数据量 "+str(len(data)))
    if len(data) > 0:
        data = data[fields]
        data.to_excel(path, index=False)
        
#合并问题数据、字典之外数据
def merge_ex_house_use(FILE_DIR,f_data):
    if os.listdir(FILE_DIR):
        ex_house_use_file_path = os.listdir(FILE_DIR)
        ex_house_use_data = get_data(
        ex_house_use_file_path, FILE_DIR)
        ex_house_use_data.index = ex_house_use_data[HOUSE_ID]
        f_data.loc[ex_house_use_data[HOUSE_ID], WH_HOUSE_USE] = ex_house_use_data[WH_HOUSE_USE]
        print('二级码已合并补充')
    else:
        print('二级码无需合补充')
    return f_data

## 城市设置

In [ ]:
CITY_CODE = 1

In [ ]:
# 城市选择
FORE_CITY = ['aaa']
if CITY_CODE == 0:
    CITY = 'b'
else:
    CITY = 'c'

## 全局设置

In [ ]:
# 城市划分
if CITY in FORE_CITY :
    DIVIDE = 'xx'
else:
    DIVIDE = 'xx'


In [ ]:
#设置输出的行数量
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# 字段设置
HOUSE_ID = '1'
HOUSE_CODE ='2'
WH_HOUSE_USE = '3'
COMMUNITY_NAME='4'
HOUSE_LOCATE = '5'
BUILD_NB='6'
ON_LAYER = '7'
ABOVE_GROUND_FLOOR='8'
BLANK_PRICE='9(元/㎡)'
HOUSE_AREA = '10'
HOUSE_NB='11'
NEW_BUILD_NB = '12'
NEW_ON_LAYER = '13'
NEW_ABOVE_GROUND_FLOOR = '14'
DISTRICT = '15'

if DIVIDE == 'zz':
    HOUSE_USE = '111'
    DEV_CITY = 'jx'
elif DIVIDE == 'xx':
    HOUSE_USE = '111'
    DEV_CITY = 'jh'

In [ ]:
# 导出字段设置
# 导出商业办公任务所需字段
COMMERCIAL_AND_OFFICE_FIELDS = [HOUSE_ID, HOUSE_CODE, HOUSE_USE, DISTRICT,
                                HOUSE_AREA, HOUSE_LOCATE, COMMUNITY_NAME, BUILD_NB, ON_LAYER, HOUSE_NB, WH_HOUSE_USE]
# 导出问题数据、非商业办公住宅数据所需字段
PROB_FIELDS = [HOUSE_ID, HOUSE_CODE, HOUSE_USE, DISTRICT,
               HOUSE_AREA, HOUSE_LOCATE, COMMUNITY_NAME, BUILD_NB, ON_LAYER, HOUSE_NB, NEW_BUILD_NB, NEW_ON_LAYER, NEW_ABOVE_GROUND_FLOOR, WH_HOUSE_USE]

# 导出幢号清洗所需字段
BUILD_NB_TASK_FIELDS = [HOUSE_ID, HOUSE_CODE, DISTRICT, HOUSE_LOCATE,
                        HOUSE_AREA, COMMUNITY_NAME, HOUSE_NB, BUILD_NB, NEW_BUILD_NB]
# 导出所在层清洗所需字段
ON_LAYER_TASK_FIELDS = [HOUSE_ID, HOUSE_CODE, DISTRICT, HOUSE_LOCATE,
                        HOUSE_AREA, COMMUNITY_NAME, HOUSE_NB, ON_LAYER, NEW_ON_LAYER]
# 导出地上层清洗所需字段
ABOVE_GROUND_TASK_FIELDS = [HOUSE_ID, HOUSE_CODE, DISTRICT, HOUSE_LOCATE, HOUSE_USE, HOUSE_AREA,
                            ON_LAYER, NEW_ON_LAYER, HOUSE_NB, COMMUNITY_NAME, BUILD_NB, NEW_BUILD_NB, ABOVE_GROUND_FLOOR, NEW_ABOVE_GROUND_FLOOR]

In [ ]:
# 日期设置
dt = datetime.datetime.now()
date = datetime.date.today()
DATE ='{}-{}-{}'.format(date.year,date.month,date.day)
EXPORT_DATE = time.strftime("%Y%m%d", time.localtime())
MONTH = str(date.month) + '月'
print('月份:', MONTH, '日期:', DATE)

In [ ]:
# 文件路径设置
#原始数据路径
R_DATA_FILE_PATH = MONTH + '/'+DATE+'/'+CITY+'/原数据/'

#人工清洗数据路径
COMMERCIAL_TASK_FILE_PATH = MONTH + '/' + DATE+'/'+CITY + '/'+CITY+'-商业数据-' +'-'+DATE+'.xlsx'
OFFICE_TASK_FILE_PATH = MONTH + '/'+DATE+'/'+CITY +  '/'+CITY+'-办公数据-' +'-'+DATE+'.xlsx'
EX_TASK_FILE_PATH = MONTH + '/'+DATE+'/'+CITY+'/' + CITY+'-非商业办公住宅数据-' +'-'+DATE+'.xlsx'
DATA_88_TASK_FILE_PATH = MONTH + '/'+DATE+'/'+CITY+'/' + CITY+'-住宅问题数据-' +'-'+DATE+'.xlsx'

#幢号、所在层、地上层
BUILD_NB_TASK_FILE_PATH = MONTH + '/'+DATE+'/'+CITY+'/'+CITY+'-幢号人工清洗.xlsx'
ON_LAYER_TASK_FILE_PATH =MONTH + '/'+DATE+'/'+CITY+'/'+CITY+'-所在层人工清洗.xlsx'
ABOVE_GROUND_FLOOR_TASK_FILE_PATH = MONTH + '/' + DATE+'/'+CITY+'/'+CITY+'-地上层人工清洗.xlsx'

#清洗结果导出
DEV_RESULT_FILE_PATH = MONTH + '/'+DATE+'/房屋清洗结果-开发-'+DATE + '/' + CITY + '/'+EXPORT_DATE+'-dsp_house_'+DEV_CITY+'_wh.xlsx'
SPLIT_DEV_RESULT_FILE_PATH =MONTH + '/'+DATE+'/房屋清洗结果-开发-'+DATE + '/' + CITY + '/'+EXPORT_DATE+'-dsp_house_'+DEV_CITY+'_wh-'
RESULT_FILE_PATH = MONTH + '/'+DATE+'/房屋清洗结果-'+DATE + '/' + CITY + '/'+CITY+'-清洗结果-'+DATE+'.xlsx'
# CONCAT_RESULT_FILE_PATH = 'H:/code/日常清洗房屋新增/清洗结果/'+CITY +\
#     '/' + str(dt.month) + '月' + '/'+CITY+'-清洗结果-'+DATE+'.xlsx'

#新建文件夹路径
MONTH_PATH = './' + MONTH
COMMERCIAL_OFFICE_RESULT_DIR = MONTH + '/'+DATE+'/'+CITY+'/'+'本次其他结果/'
PROB_RESULT_DIR = MONTH + '/' + DATE+'/'+CITY+'/'+'问题数据/'
BUILD_NB_DIR = MONTH + '/'+DATE+'/'+CITY+'/'+'build_nb/'
ON_LAYER_DIR =MONTH + '/'+DATE+'/'+CITY+'/'+'on_layer/'
AB_FLOOR_DIR =MONTH + '/'+DATE+'/'+CITY+'/'+'ab_floor/'
RESULT_DIR =MONTH + '/'+DATE+'/房屋清洗结果-'+DATE+'/' + CITY + '/'
DEV_RESULT_DIR = MONTH + '/'+DATE+'/房屋清洗结果-开发-'+DATE+'/' + CITY + '/'
# CONCAT_RESULT_DIR = 'H:/code/日常清洗房屋新增/清洗结果/'+CITY+'/' + str(dt.month) + '月'  # 存放合并数据的文件夹

DIRS = [MONTH_PATH, COMMERCIAL_OFFICE_RESULT_DIR, PROB_RESULT_DIR,
        BUILD_NB_DIR, ON_LAYER_DIR, AB_FLOOR_DIR, RESULT_DIR, 
        R_DATA_FILE_PATH, DEV_RESULT_DIR]

# 批量创建文件夹
for output_dir in DIRS:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [ ]:
#房屋用途划分
if DIVIDE == 'xx':
    ghyt_10  = ['111']
    ghyt_123 = ['111']
    ghyt_126 =['111']
    ex_ghyt = ghyt_1+ghyt_12+ghyt_16
elif DIVIDE == '11':
    ghyt_10  = ['111']
    ghyt_123 = ['3111']
    ghyt_126 =['111']
    ex_ghyt = ['111']    

# 数据整理

## 导入数据

In [ ]:
r_data = get_r_data(R_DATA_FILE_PATH)
len(r_data)

In [ ]:
# 拷贝原始数据
f_data = r_data.copy()
f_data = f_data[f_data[HOUSE_CODE].notnull()]
f_data.index = f_data[HOUSE_ID]
f_data[WH_HOUSE_USE] = np.NaN
f_data

In [ ]:
f_data[HOUSE_USE].groupby(f_data[HOUSE_USE]).count()

## 商办数据导出

In [ ]:
data_14 = f_data[f_data[HOUSE_USE].isin(ghyt_14)].copy()
data_18 = f_data[(f_data[HOUSE_USE].isin(ghyt_18))].copy()

In [ ]:
output_ex_data(data_14,COMMERCIAL_AND_OFFICE_FIELDS, COMMERCIAL_TASK_FILE_PATH, '商业')
output_ex_data(data_18,COMMERCIAL_AND_OFFICE_FIELDS, OFFICE_TASK_FILE_PATH, '办公')

## 住宅数据处理

In [ ]:
data_1 = f_data[f_data[HOUSE_USE].isin(ghyt_1)].copy()
len(data_1)

# 数据清洗

## 房屋小区名称清洗

In [ ]:
def parse_community_name(x):
    if type(x) == str:
        x = x.strip()
        r_path = '(.*?)[\d|A-Z|a-z、~-]+?(幢|号楼|#|室).*?'
        r = re.compile(r_path, re.S).search(x)
        left_path = '(衢州市|衢江区|柯城区|湖州市|吴兴区|南浔区|台州市|黄岩区|椒江区|绍兴市|嵊州市|台州市|玉环市|黄岩区|路桥区|温岭市|嘉兴市|南湖区|秀洲区|桐乡市|街道|镇|县)(.*)'
        left_special = re.compile(left_path, re.S).search(x)
        num_end = '(.*)\d+$'
        num = re.compile(num_end, re.S).search(x)
        if r:
            if len(r.group(1)) > 0:
                left = re.compile(left_path, re.S).search(r.group(1))
                if left:
                    if len(left.group(2)) > 0:
                        return left.group(2)
                return r.group(1)
        if left_special:
            if len(left_special.group(2)) > 0:
                return left_special.group(2)
        if num:
            if len(num.group(1)) > 0:
                return num.group(1)
        return x

In [ ]:
data_10['房屋小区名称'] = data_10[COMMUNITY_NAME].apply(parse_community_name)
data_10['房屋小区名称'] = data_10.loc[:, '房屋小区名称'].combine_first(
    data_10[HOUSE_LOCATE].apply(parse_community_name))
data_10['房屋小区名称'] = data_10.loc[:, '房屋小区名称'].combine_first(
    data_10['房屋小区名称'].apply(parse_community_name))
data_10['房屋小区名称'].drop_duplicates()

In [ ]:
data_10[data_10['房屋小区名称'] == '']

## 幢号清洗

In [ ]:
def parse_unit(x):
    if type(x) == str:
        x = x.strip()
        path = re.compile('.*?([A-Z|\d+]+?)(幢|号楼|#).*?')
        r = path.search(x)
        if r:
            return r.group(1)
    return np.NaN

In [ ]:
data_10.loc[:, '幢号-清洗后'] = data_10[HOUSE_LOCATE].apply(parse_unit)
data_10.loc[:, '幢号-清洗后'] = data_10['幢号-清洗后'].combine_first(data_10[BUILD_NB].apply(parse_unit))
data_10['幢号-清洗后'] = data_10['幢号-清洗后']
data_10['幢号-清洗后'].drop_duplicates()

In [ ]:
# 导出幢号
build_nb = data_10[data_10[BUILD_NB] != data_10['幢号-清洗后']][[HOUSE_ID, HOUSE_CODE, DISTRICT, COMMUNITY_NAME, HOUSE_LOCATE, HOUSE_USE, HOUSE_AREA,
                                                          BLANK_PRICE,BUILD_NB, HOUSE_NB, '幢号-清洗后']]
if build_nb[BUILD_NB].isnull().all():
    print('原数据幢号为空')
else:
    output_ex_data(build_nb,BUILD_NB_TASK_FIELDS, BUILD_NB_TASK_FILE_PATH, '幢号')
build_nb.head(5)

In [ ]:
# 更新清洗后的幢号
update_code(BUILD_NB_DIR,data_10,'幢号-清洗后')
data_10['幢号-清洗后'].drop_duplicates()

## 所在层清洗

In [ ]:
# 所在层清洗
def split_on_layer(x):
    x = str(x)
    path = re.compile('.*?(\d+)[0-9](\d+?)室')
    r = path.search(x)
    if r:
        return r.group(1)
    return np.NaN


def parse_on_layer(x):
    x = str(x)
    exist_number = re.search(r'\d', x)
    if exist_number:
        all_nums = re.findall(r'\d+', x, re.S)
        max_layer = max(all_nums)
        return max_layer
    return np.NaN


def split_house_nb(x):
    x = str(x)
    path = re.compile('.*?(\d\d?)\d\d$')
    r = path.search(x)
    if r:
        return r.group(1)
    return np.NaN


data_10['所在层-纯数'] = data_10[HOUSE_LOCATE].apply(split_on_layer)
data_10.loc[:, '所在层-纯数'] = data_10['所在层-纯数'].combine_first(
    data_10[ON_LAYER].apply(parse_on_layer))
data_10.loc[:, '所在层-纯数'] = data_10['所在层-纯数'].combine_first(
    data_10[HOUSE_NB].apply(split_house_nb))
data_10.loc[:, '所在层-纯数'] = data_10['所在层-纯数'].apply(parse_on_layer)
data_10.loc[:, '所在层-纯数'] = data_10.loc[:, '所在层-纯数']
# data_10['所在层-纯数'].drop_duplicates()

In [ ]:
# 导出所在层
data_10[ON_LAYER] = data_10[ON_LAYER].apply(remove_space)
on_layer = data_10[data_10[ON_LAYER] != data_10['所在层-纯数']][[HOUSE_ID,HOUSE_CODE, DISTRICT, COMMUNITY_NAME, HOUSE_LOCATE, HOUSE_USE, HOUSE_AREA,
                                                          BLANK_PRICE,BUILD_NB, HOUSE_NB, ON_LAYER,'所在层-纯数']]

if on_layer[ON_LAYER].isnull().all():
    print('不一致数据原数据所在层为空')
else:
    output_ex_data(on_layer,ON_LAYER_TASK_FIELDS, ON_LAYER_TASK_FILE_PATH, '所在层')
on_layer

In [ ]:
# 更新人工后的所在层
update_code(ON_LAYER_DIR,data_10,'所在层-纯数')
data_10['所在层-纯数'].drop_duplicates()

## 地上层清洗

In [ ]:
def parse_above_ground_layer(data):
    floor_data = data[data['所在层-纯数'].notnull()].copy()
    floor_data['所在层-纯数'] = floor_data['所在层-纯数'].apply(remove_space)   
    floor_data['所在层-纯数'] = floor_data['所在层-纯数'].astype(int)
    max_floor = floor_data.groupby(['房屋小区名称', '幢号-清洗后'])['所在层-纯数'].max()
    max_floor = pd.DataFrame(max_floor).reset_index()
    max_floor.columns = ['房屋小区名称', '幢号-清洗后', '地上层-清洗后']
    data = pd.merge(data, max_floor, on=['房屋小区名称', '幢号-清洗后'], how='outer')
#     data.loc[:, '地上层'] = data['地上层'].astype(str)
    data.index = data[HOUSE_ID]
    return data

data_10 = parse_above_ground_layer(data_10)

In [ ]:
# 导出地上层-清洗和原字段值不一致数据
data_10[ABOVE_GROUND_FLOOR] = data_10[ABOVE_GROUND_FLOOR].apply(remove_space)
ab_floor_diff = data_10[data_10[ABOVE_GROUND_FLOOR] != data_10['地上层-清洗后'].astype(str)][[HOUSE_ID,HOUSE_CODE, DISTRICT, COMMUNITY_NAME, HOUSE_LOCATE, HOUSE_USE, HOUSE_AREA,
                                                          BLANK_PRICE,BUILD_NB ,'幢号-清洗后',HOUSE_NB, ON_LAYER,'所在层-纯数','房屋小区名称',ABOVE_GROUND_FLOOR,'地上层-清洗后']]
if ab_floor_diff[ABOVE_GROUND_FLOOR].isnull().all():
    print('原数据地上层为空')
else:
#     output_data(ab_floor_diff,ABOVE_GROUND_FLOOR_TASK_FILE_PATH)
    output_ex_data(ab_floor_diff,ABOVE_GROUND_TASK_FIELDS, ABOVE_GROUND_FLOOR_TASK_FILE_PATH, '地上层')
ab_floor_diff.head(5)

In [ ]:
# 更新人工比对后的地上层
update_code(AB_FLOOR_DIR,data_10,'地上层-清洗后')
data_10['地上层-清洗后'].drop_duplicates()

## 更新房屋小区名称、幢号、地上层、所在层

In [ ]:
f_data.loc[:,'xx'] = np.NaN

## 房屋打码

### 地上层为空的数据（问题数据）

In [ ]:
nan_10_data = data_10[data_10['地上层-清洗后'].isnull()].copy()
nan_10_data.loc[:,WH_HOUSE_USE] = 'v'
len(nan_10_data)

In [ ]:
update_table(nan_10_data)
# f_data[f_data[WH_HOUSE_USE].notnull()]

### 地上层不为空的数据

In [ ]:
re_ex_10_data = data_10[data_10['地上层-清洗后'].notnull()].copy()
re_ex_10_data.loc[:, '地上层-清洗后'] = re_ex_10_data['地上层-清洗后'].apply(int)
re_ex_10_data.groupby('地上层-清洗后')['地上层-清洗后'].count()

In [ ]:
# 根据地上层标注
dsc = re_ex_10_data['地上层-清洗后']
re_ex_10_data.loc[dsc <= 0, WH_HOUSE_USE] = '1'
re_ex_10_data.loc[(dsc > 0) & (dsc <= 3), WH_HOUSE_USE] = '2'
re_ex_10_data.loc[(dsc > 3) & (dsc <= 7), WH_HOUSE_USE] = '3'
re_ex_10_data.loc[(dsc > 7) & (dsc <= 11), WH_HOUSE_USE] = '4'
re_ex_10_data.loc[(dsc > 11) & (dsc <= 33), WH_HOUSE_USE] = '5'
re_ex_10_data.loc[(dsc > 33),WH_HOUSE_USE] = '6'
update_table(re_ex_10_data)
len(f_data[f_data[WH_HOUSE_USE].notnull()])

## 字典值比对

### 对应杭州字典

In [ ]:
if DIVIDE == 'x':
    code = pd.read_excel('房屋字典'+'/'+CITY+'字典'+'.xlsx', sheet_name='脚本')
    code['杭州'] = code['杭州'].astype(str)
    code[CITY] = code[CITY].astype(str)
    code.replace('nan', np.NaN, inplace=True)
    code_dict = dict(zip(code[CITY], code['杭州']))
else:
    pass

### 查询不包含的字典值

In [ ]:
def preproccess_code(code):
    for key, value in code_dict.items():
        if code == key:
            return value

In [ ]:
if DIVIDE == 'x':
    house_use_notnull = r_data[r_data.loc[:,
                                      HOUSE_USE].notnull()].copy()
    house_use_notnull.loc[:, WH_HOUSE_USE] = house_use_notnull[HOUSE_USE].apply(
    preproccess_code)
    house_use_notnull[house_use_notnull[WH_HOUSE_USE].isnull()
                  ][HOUSE_USE].drop_duplicates()

In [ ]:
if DIVIDE == 'd':
    house_use_notnull = r_data[r_data.loc[:,
                                          HOUSE_USE].notnull()].copy()
    house_use_notnull.loc[:, WH_HOUSE_USE] = house_use_notnull[HOUSE_USE].apply(
        preproccess_code)
    not_exist_code = house_use_notnull[house_use_notnull[WH_HOUSE_USE].isnull(
    )][HOUSE_USE].drop_duplicates()
    df_not_exist_code = pd.DataFrame(not_exist_code)
    df_not_exist_code = df_not_exist_code[~df_not_exist_code[HOUSE_USE].isin(
        ex_ghyt_10_123_126)]
    df_not_exist_code

In [ ]:
if DIVIDE == 'x':
    house_use_notnull[house_use_notnull[WH_HOUSE_USE].isnull()][HOUSE_USE].drop_duplicates()

### 更新二级码

In [ ]:
if DIVIDE == 'ew':
    update_table(house_use_notnull)
    f_data[f_data[WH_HOUSE_USE].notnull()]

## 字典之外的数据处理

In [ ]:
if DIVIDE == 'g':
    not_exist_code_data = f_data[f_data[HOUSE_USE].isin(
        df_not_exist_code[HOUSE_USE])].copy()
    output_ex_data(not_exist_code_data, PROB_FIELDS, EX_TASK_FILE_PATH, '字典之外')
elif DIVIDE == 'g':
    not_exist_code_data = f_data[~f_data[HOUSE_USE].isin(ex_ghyt_10_123_126)]
    output_ex_data(not_exist_code_data,PROB_FIELDS,  EX_TASK_FILE_PATH, '字典之外')

## 问题数据处理

In [ ]:
data_88 = f_data[f_data[WH_HOUSE_USE] == 'sd']
output_ex_data(data_88, PROB_FIELDS, DATA_88_TASK_FILE_PATH, '问题数据')

# 合并清洗结果

## 合并补充二级码商业办公数据

In [ ]:
f_data = merge_ex_house_use(COMMERCIAL_OFFICE_RESULT_DIR,f_data)
f_data['房屋用途'] = f_data['房屋用途'].astype(str)
f_data['房屋用途'].groupby(f_data['房屋用途']).count()

## 合并字典之外的数据

In [ ]:
f_data = merge_ex_house_use(PROB_RESULT_DIR,f_data)
f_data['房屋用途'] = f_data['房屋用途'].astype(str)
# f_data['房屋用途'].groupby(f_data['房屋用途']).count()

## 导出清洗结果

### 导出本地存储清洗结果

In [ ]:
def save_data(f_data):
    f_data.to_excel(RESULT_FILE_PATH,index=False)
    print('此次清洗数量为：',len(f_data))
save_data(f_data)
f_data['房屋用途'].groupby(f_data['房屋用途']).count()

In [ ]:
print('房屋用途为空数据量为：',len(f_data[f_data['房屋用途'].isnull()]), '条')

### 导出平台维护结果

In [ ]:
# 平台导入所需字段
import_fields = ['xxxxx']

In [ ]:
# 1000条以上数据做拆分
upper_limit = 1400
single_quantity = 700
if len(f_data) < upper_limit:
    f_data[import_fields].to_excel(DEV_RESULT_FILE_PATH, index=False)
else:
    f_data[import_fields].to_excel(DEV_RESULT_FILE_PATH, index=False)
    for i in range(math.ceil(len(f_data)/single_quantity)):
        f_data[import_fields][i*single_quantity: (i+1)*single_quantity].to_excel(
            SPLIT_DEV_RESULT_FILE_PATH + str(i) + '.xlsx', index=False)